In [1]:
import sys
import os
import pandas as pd
import json

from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath('../'))

## Proof Net

`hoskinson-center/proofnet` is in LEAN 3, we use the LEAN 4 translation used in the paper *DeepSeek Prover*

Not much discrete math, maybe subset `Ireland-rosen` or `Rudin`?

In [2]:
df = pd.read_json('proofnet.jsonl', lines=True)

In [3]:
df['name'] = 'f2f_' + df['name']
df = df.drop(columns=['goal'])

In [13]:
rosen_df = df[df['name'].str.contains('rosen')]
print(rosen_df)

Empty DataFrame
Columns: [name, split, informal_prefix, formal_statement, header]
Index: []


In [9]:
df.head()

,name,split,informal_prefix,formal_statement,header
0,f2f_exercise_1_13a,valid,/-- Suppose that $f$ is holomorphic in an open...,theorem exercise_1_13a {f : ℂ → ℂ} (Ω : Set ℂ)...,import Mathlib\n\nopen Complex Filter Function...
1,f2f_exercise_1_13b,test,/-- Suppose that $f$ is holomorphic in an open...,theorem exercise_1_13b {f : ℂ → ℂ} (Ω : Set ℂ)...,import Mathlib\n\nopen Complex Filter Function...
2,f2f_exercise_1_13c,valid,/-- Suppose that $f$ is holomorphic in an open...,theorem exercise_1_13c {f : ℂ → ℂ} (Ω : Set ℂ)...,import Mathlib\n\nopen Complex Filter Function...
3,f2f_exercise_1_19a,test,/-- Prove that the power series $\sum nz^n$ do...,theorem exercise_1_19a (z : ℂ) (hz : abs z = 1...,import Mathlib\n\nopen Complex Filter Function...
4,f2f_exercise_1_19b,valid,/-- Prove that the power series $\sum zn/n^2$ ...,theorem exercise_1_19b (z : ℂ) (hz : abs z = 1...,import Mathlib\n\nopen Complex Filter Function...


In [5]:
print(df.iloc[0].informal_prefix)

/-- Suppose that $f$ is holomorphic in an open set $\Omega$. Prove that if $\text{Re}(f)$ is constant, then $f$ is constant.-/



In [6]:
print(df.iloc[0].formal_statement)

theorem exercise_1_13a {f : ℂ → ℂ} (Ω : Set ℂ) (a b : Ω) (h : IsOpen Ω)
  (hf : DifferentiableOn ℂ f Ω) (hc : ∃ (c : ℝ), ∀ z ∈ Ω, (f z).re = c) :
  f a = f b :=


In [14]:
topics = set()
for i in range(len(df)):
    topics.update(df.iloc[i]['header'].split('\n'))

for topic in topics:
    print(topic)


open scoped BigOperators
| 0 => sqrt 2
  generateFrom {S : Set X | ∃ a b, a < b ∧ S = Ico a b}
def countably_compact (X : Type*) [TopologicalSpace X] :=
def rational (x : ℝ) := x ∈ range ((↑) : ℚ → ℝ)
  ∀ U : Set X, Infinite U → ∃ x ∈ U, ClusterPt x (𝓟 U)
noncomputable section
def lower_limit_topology (X : Type) [Preorder X] :=
  (∀ i, IsOpen (U i)) ∧ ((univ : Set X) ⊆ ⋃ i, U i) →
  (∃ t : Finset ℕ, (univ : Set X) ⊆ ⋃ i ∈ t, U i)
open Topology Filter Real Complex TopologicalSpace Finset
| (n + 1) => sqrt (2 + sqrt (f n))
open scoped BigOperators Topology
open Fintype Set Real Ideal Polynomial
  ∀ U : ℕ → Set X,
  univ ∈ T ∧
import Mathlib
  ({S : Set ℝ | ∃ a b, a < b ∧ S = Ioo a b} ∪ {S : Set ℝ | ∃ a b, a < b ∧ S = Ioo a b \ K})
open Complex Filter Function Metric Finset
def K : Set ℝ := {r | ∃ n : ℕ, r = 1 / n}
noncomputable def f : ℕ → ℝ
def g (n : ℕ) : ℝ := sqrt (n + 1) - sqrt n
set_option checkBinderAnnotations false
open Filter Real Function
open RingHom
--center of (G × H) equiv

In [8]:
stop

NameError: name 'stop' is not defined

## Putnam Bench (110)

In [5]:
ds = load_dataset("amitayusht/PutnamBench")

Repo card metadata block was not found. Setting CardData to empty.


In [6]:
df = pd.DataFrame(ds['train'])

In [7]:
df = df.drop(columns=['coq_statement', 'isabelle_statement', 'informal_solution'])
df = df.dropna()

In [8]:
# get unique values from the column 'tags'
all_tags = set()

for tags in df.tags.unique():
    tags = tags.replace('[', '').replace(']', '').replace('\'', '')
    tags = tags
    tag_list = tags.split(',')
    for tag in tag_list:
        all_tags.add(tag.strip())

print(all_tags)

{'algebra', 'abstract_algebra', 'geometry', 'probability', 'analysis', 'combinatorics', 'set_theory', 'linear_algebra', 'number_theory'}


We will keep `{'set_theory', 'probability', 'number_theory', 'combinatorics'}`

In [19]:
wanted_tags = ['set_theory', 'probability', 'number_theory', 'combinatorics']
putnam_df = df[df.tags.str.contains('|'.join(wanted_tags))]


In [20]:
putnam_df

,name,lean4_statement,informal_statement,tags
0,putnam_1981_a1,abbrev putnam_1981_a1_solution : ℝ := sorry\n-...,Let $E(n)$ be the greatest integer $k$ such th...,"['analysis', 'number_theory']"
5,putnam_1981_b3,theorem putnam_1981_b3\n(P : ℕ → Prop := fun n...,"Prove that, for infinitely many positive integ...",['number_theory']
10,putnam_2007_a4,abbrev putnam_2007_a4_solution : Set (Polynomi...,A \emph{repunit} is a positive integer whose d...,"['analysis', 'algebra', 'number_theory']"
17,putnam_1980_a2,abbrev putnam_1980_a2_solution : ℕ → ℕ → ℕ := ...,Let $r$ and $s$ be positive integers. Derive a...,['number_theory']
24,putnam_1980_b4,theorem putnam_1980_b4\n{T : Type}\n(X : Finse...,Let $X$ be a finite set with at least $10$ ele...,"['set_theory', 'combinatorics']"
...,...,...,...,...
502,putnam_1993_b5,theorem putnam_1993_b5\n(pdists : (Fin 4 → (Fi...,Show there do not exist four points in the Euc...,"['geometry', 'number_theory', 'linear_algebra']"
504,putnam_2015_a2,abbrev putnam_2015_a2_solution : ℕ := sorry\n-...,"Let $a_0=1$, $a_1=2$, and $a_n=4a_{n-1}-a_{n-2...",['number_theory']
505,putnam_2015_a3,abbrev putnam_2015_a3_solution : ℂ := sorry\n-...,Compute $\log_2 \left( \prod_{a=1}^{2015}\prod...,"['number_theory', 'algebra']"
507,putnam_2015_a5,theorem putnam_2015_a5\n(q : ℕ)\n(Nq : ℕ)\n(qo...,"Let $q$ be an odd positive integer, and let $N...",['number_theory']


## Mini F2F